In [5]:
from nltk.corpus import wordnet as wn
all_nouns = list(wn.all_synsets('n'))

In [31]:
import numpy as np
import cPickle as pickle
word_embedding_matrix = np.load('/u/vendrov/qanda/glove/glove_matrix.npy')
word2index = pickle.load(open('/u/vendrov/qanda/glove/glove_word_index.pkl', 'r'))

In [32]:
import numpy as np
word_embeddings = np.zeros((len(all_nouns), word_embedding_matrix.shape[1]))
in_w2v = np.zeros((len(all_nouns),))
for i in range(len(all_nouns)):
    synset = all_nouns[i]
    # the embedding of the synset is the embedding of the first lemma which is in word2vec
    for lemma in synset.lemma_names():
        if lemma in word2index:
            word_embeddings[i,:] = word_embedding_matrix[word2index[lemma]]
            in_w2v[i] = True
        words = lemma.split('_')
        if all(map(lambda l: l in word2index, words)):
            # take mean of component words
            for word in words:
                word_embeddings[i] += word_embedding_matrix[word2index[word]] / len(words)
            in_w2v[i] = True
        if in_w2v[i]: break
    
    if not in_w2v[i]:
        print("None of these lemma names are in the vocabulary: ")
        print(synset.lemma_names())

None of these lemma names are in the vocabulary: 
[u'biont']
None of these lemma names are in the vocabulary: 
[u'deviationism']
None of these lemma names are in the vocabulary: 
[u'footfault']
None of these lemma names are in the vocabulary: 
[u'viatication', u'viaticus']
None of these lemma names are in the vocabulary: 
[u'finger-roll']
None of these lemma names are in the vocabulary: 
[u'countershot']
None of these lemma names are in the vocabulary: 
[u"fielder's_choice"]
None of these lemma names are in the vocabulary: 
[u'ballottement']
None of these lemma names are in the vocabulary: 
[u"bull's_eye"]
None of these lemma names are in the vocabulary: 
[u'anathematization', u'anathematisation']
None of these lemma names are in the vocabulary: 
[u'chance-medley']
None of these lemma names are in the vocabulary: 
[u'uxoricide']
None of these lemma names are in the vocabulary: 
[u'felo-de-se']
None of these lemma names are in the vocabulary: 
[u'cardiospasm']
None of these lemma names 

In [41]:
### get hypernyms

# get mapping of synset id to index
id2index = {}
for i in range(len(all_nouns)):
    id2index[all_nouns[i].name()] = i
    
# get hypernym relations
hypernyms = []
for synset in all_nouns:
    for h in synset.hypernyms() + synset.instance_hypernyms():
        hypernyms.append([id2index[synset.name()], id2index[h.name()]])
hypernyms = np.array(hypernyms)

Synset('physical_entity.n.01')

In [39]:
import h5py
f = h5py.File('wordnet.h5')

In [46]:
f.create_dataset('embeddings', data=word_embeddings)
f.create_dataset('in_w2v', data=in_w2v)
f.create_dataset('hypernyms', data = hypernyms)

<HDF5 dataset "hypernyms": shape (84505, 2), type "<i8">

In [47]:
f.close()

In [48]:
# save list of synset names
names = map(lambda s: s.name(), all_nouns)
import json
json.dump(names, open('synset_names.json', 'w'))

In [76]:
numpy.linalg.norm(word_embeddings[14])

0.0